Frameworks for ML scaling and production
----

# Hadoop

https://hadoop.apache.org/

## Introduction and Use Case

Hadoop is an open-source distributed data management system. It combines tools to store, analyze, and process large-scale pools of data on clusters of servers, without requiring specialized hardware. The "vanilla" version maintained by the Apache Foundation is quite intricate and not entirely stable, so there are many commercial distributions offered by third parties (such as Cloudera, Hortonworks). The major cloud services ([Google](https://cloud.google.com/dataproc?hl=en), Amazon, Microsoft) can also host Hadoop, either with their own out-of-the-box solutions or provided by commercial distributions.

This [table](https://hadoopecosystemtable.github.io/) summarizes libraries and applications within the Hadoop "ecosystem," including those produced by Apache itself and many others.

Cloud data systems like Hadoop represent an alternative to relational databases in order to provide greater scalability and speed at large scales. It is often said that databases can optimize on 2 of 3 goals (CAP): consistency, availability, and partitioning (i.e. scalability). SQL priortizes C and A, while Hadoop prioritizes A and P. Because it lacks the transaction control of relational databases, it is better suited to "behavioral" rather than "line of business" data (such as customer accounts, supply chains, etc). Behavioral data is collected *in aggregate* as side-effect of user activity. Rather than being tracked and queried on the level of individuals, this data is primarily useful for the general patterns than can be seen in it -- hence it is acceptable to deprioritize consistency in a way that would not be workable for business-critical data.


## Alternatives for Running Hadoop

1. Apache Hadoop open source versus vendor services
1. Docker images versus virtual machines
1. Local file system, pseudo-distributed, fully distributed on own servers, versus on the cloud
1. Versioning: Apache Hadoop updates frequently, and there are incompatbilities with some versions. MapReduce in particular went through a major 1.0 to 2.0 transition.

## Elements of the Hadoop Ecosystem

### Hadoop File System (HDFS)

Developed out of a system published by Google, HDFS promises scalability on "commodity" hardware. By default, it employs 3x data redundancy and enables larger chunk sizes than other formats. It is also possible to use the native file systems of cloud services.

HDFS is immutable: any operations on data are saved as new files in the system rather than altering existing data. This includes re-executing operations: by default this will generate new outputs files every time instead of overwriting.

The HDFS command-line interface syntax is `hadoop fs -command` (or sometimes `dfs`) where `command` shares many Linux shell commands like `cat`, `mkdir`, `ls` etc plus distinctive commands like `put` and `get` to moves file betweens HDFS and other storage (local/cloud). HDFS locations are written as urls `hdfs://...`

### MapReduce

The distributed processing framework for Hadoop. Implemented in Java, MapReduce processes (and anything else executing on a Hadoop server) are executed in Java virtual machines (JVM). Each process is a distinct VM that does not share state. The quirk this introduces is that although the syntax is object-oriented (being Java, everything is a class, in this case Static classes), the paradigm is much closer to functional programming, as each process can only take in data and output results without being able to reference the results of other parallel instances.

There are now also APIs for languages more commonly used in data science like Python and R, as well as interfaces for other systems programming languages like C# and C++.

The basic unit of a MapReduce routine is the **Job**, which is instanciated to carry out Map and Reduce operations on data. The **Map** functionality applies some set of operations *on each node* in the Hadoop cluster. It returns a set of key/value pairs. The **Reduce** functionality aggregates key/value pairs (on some subset of nodes) and returns a combined list, which is stored as a new file in the system. In between these two steps, the data (duplicated across nodes) is "shuffled and sorted" to processing nodes. For efficiency, it is possible to do a preliminary **Combine** stage on the original node, so as to increase the density of data that needs to be transferred across nodes for sorting and later reduction.

So, for example, a basic word count operation -- producing a list of the unique words in a text and their corresponding counts -- the map function would turn the text into a list of words (each with 1 instance) and the reduce function would take look at each word and sum up the instances.

It is considered good practice to subdivide tasks so that each routine performs only a single operation, and more complex operations are the result of chains of jobs. Pre-processing, for example, can be run as a "map only" job.

Jobs are run by submitting them to the scheduler: this takes the form of indicating a `.jar` file and the class name to run as main, plus needed arguments like source and output locations. From the command line, the syntax is `hadoop jar filename.jar input output`.

MapReduce 1.0 was limited because it could only process in batch and was not easy to customize. The 2.0 update allows more "on-time" operations and more intricate controls of how operations are carried out.

### YARN

Yet Another Resource ____: an abstraction layer added along with MapReduce 2.0 that allows a wider range of data processing on top of HDFS.


### Apache Spark

An alternative for distributed data processing engine, which primarily operates in memory.

### HBase

A wide-column, schema-on-read (NoSQL) database format that acts as a relatively accessible front-end to data stored in a Hadoop cluster.

### Hive

A query language interface for HBase, which acts as a MapReduce front-end, also known as HQL or H-SQL. The syntax is similar to SQL, but backend is fundamentally different. For one thing because it is a front-end for MapReduce (via often HBase), it is executing batch jobs on the cluster, which can take substantial time.

`CREATE TABLE` commands pull requested fields from data into a wide table, then `SELECT...WHERE` commands can pull out specific records. NB, since the underlying data is not relational, `JOIN` statements are often impractical.

### Pig

A scripting tool for Hadoop, used especially for data input and cleaning (ETL: extract, transform, load). Its native language is called Pig Latin.

### Oozie

A workflow manager used to coordinate scripts from multiple libraries. Jobs are scripted using XML, so commercial GUIs are often used in practice.

### Sqoop

Command-line utility for transferring data between relation databases and Hadoop clusters. 

### ZooKeeper

Centralized service for Hadoop configuration information, to create ensembles of programs. It performs computation in-memory for more real-time operations.

## MapReduce

